# 🔄 Day 1-00.03: RAFT 전처리하기 (초보자용)

## 🎯 이번 노트북에서 할 일
- **RAFT 방법론** 이해하기
- **Distractor Documents** 포함한 훈련 데이터 생성
- **데이터를 파인튜닝용으로 변환**하기
- **최종 데이터 저장**하기

## 💡 RAFT란?
**Retrieval Augmented Fine Tuning**의 줄임말로, 도메인 특화 RAG 성능을 향상시키는 파인튜닝 기법입니다.

### 🔧 RAFT의 핵심 아이디어
1. **"Open Book" 시험**에 비유: 모델이 문서를 참조하면서 답변
2. **Distractor Documents**: 관련 없는 방해 요소 문서들을 무시하도록 학습
3. **올바른 시퀀스 인용**: 관련 문서에서 직접 인용하여 답변 생성
4. **도메인 특화 지식**: 특정 도메인의 문서 집합에 특화된 학습

### 🎯 RAFT의 목적
- **문서 필터링**: 관련 있는 문서와 관련 없는 문서를 구분
- **인용 기반 답변**: 관련 문서에서 직접 인용하여 정확한 답변 생성
- **도메인 적응**: 특정 도메인의 문서 집합에 특화된 RAG 성능 향상

### 📝 RAFT 훈련 데이터 예시
```
질문: 인공지능이란 무엇인가요?

문서 1: 인공지능(AI)은 컴퓨터가 인간의 지능을 모방하는 기술입니다. [관련 문서]
문서 2: 요리는 재료를 조리하여 맛있는 음식을 만드는 기술입니다. [Distractor]
문서 3: 자동차는 바퀴가 달린 교통수단입니다. [Distractor]

답변: 인공지능(AI)은 컴퓨터가 인간의 지능을 모방하는 기술입니다. [문서 1에서 인용]
```

1️⃣ RAFT의 기본 아이디어

RAG(Retrieval-Augmented Generation)는 질문 → 검색된 문서(context) → 답변 흐름을 전제로 하죠.

하지만 실제 서비스 환경에서는 항상 “좋은 문서”가 검색되는 게 아닙니다.

관련 문서가 전혀 안 잡히거나,

잡히긴 했지만 엉뚱한 내용만 있을 수 있습니다.

그래서 RAFT 학습 시에는 “관련 문서가 없는 경우에도 어떻게 답할지”를 모델이 경험하게 합니다.

2️⃣ 왜 관련 문서 없이도 답을 주게 만드나?

현실적 시뮬레이션

실제 운영 환경에서 retriever가 실패할 확률은 꽤 높습니다.

이때 모델이 “모르겠습니다”라고 하든, 또는 일반 지식으로 답변을 보완하든 일관된 행동을 해야 합니다.

모델의 fallback 전략 학습

모델이 context에 전적으로 의존하게만 학습하면 → 문서가 없을 때 완전히 무력화됩니다.

그래서 “문서가 없으면 일반 지식으로 답하기” 또는 “문서가 없으면 모른다고 말하기” 중 하나를 학습시키는 겁니다.

Hallucination 제어

관련 없는 문서가 들어왔을 때도 억지로 맞춰서 답하는 대신,

“이 문서는 질문과 관련이 없습니다. 따라서 답변할 수 없습니다.”라는 식으로 행동하도록 fine-tuning 합니다.

3️⃣ 샘플링 방식 (간단 예시)

Positive sample: 관련 문서 + 질문 + 정답

Negative sample: 무관한 문서 + 질문 + (정답 or “없음”)

No-context sample: 문서 없이 질문만 + 정답

이렇게 섞어서 학습시키면,

모델은 문서가 있을 때는 근거를 활용하고,

문서가 없거나 무관할 때는 fallback 동작(“없음” 또는 일반 지식 활용)을 익히게 됩니다.


## 1. 필요한 라이브러리 불러오기


In [ ]:
# RAFT 전처리에 필요한 라이브러리들을 불러옵니다
import json
import pandas as pd
import numpy as np
from datasets import Dataset, load_from_disk
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

print("✅ RAFT 전처리 라이브러리가 준비되었습니다!")


## 2. 이전 단계에서 저장한 데이터 불러오기


In [ ]:
# 이전 단계에서 저장한 데이터를 불러옵니다
print("📥 이전 단계 데이터를 불러오는 중...")

# 데이터셋 불러오기
dataset = load_from_disk("data/rag_dataset")

# 통계 정보 불러오기
with open("data/dataset_stats.json", "r", encoding="utf-8") as f:
    stats = json.load(f)

print(f"✅ 데이터 로드 완료!")
print(f"📊 총 데이터 개수: {len(dataset):,}개")
print(f"📝 평균 질문 길이: {stats['avg_question_length']:.1f} 글자")
print(f"💬 평균 답변 길이: {stats['avg_answer_length']:.1f} 글자")
print(f"📄 평균 문서 개수: {stats['avg_context_count']:.1f}개")


## 3. RAFT 템플릿 만들기


In [ ]:
# RAFT 훈련 데이터를 생성하는 함수들
import random

def create_raft_training_sample(question, answer, context, all_contexts, num_distractors=2):
    """
    RAFT 훈련 샘플 생성: 관련 문서 + Distractor 문서들 + 인용 기반 답변
    
    Args:
        question: 사용자 질문
        answer: 정답
        context: 관련 문서 (문자열)
        all_contexts: 전체 데이터의 모든 문서들
        num_distractors: Distractor 문서 개수
    
    Returns:
        RAFT 훈련 샘플 딕셔너리
    """
    # 관련 문서는 이미 하나의 문자열
    relevant_doc = context
    
    # Distractor 문서 선택 (현재 문서와 다른 것들)
    distractor_docs = []
    for ctx in all_contexts:
        if ctx != relevant_doc and len(distractor_docs) < num_distractors:
            distractor_docs.append(ctx)
    
    # 모든 문서 결합 (관련 문서 + Distractor 문서)
    all_docs = [relevant_doc] + distractor_docs
    random.shuffle(all_docs)  # 문서 순서를 랜덤하게 섞기
    
    # 문서들을 하나의 컨텍스트로 결합
    context_text = "\n\n".join(all_docs)
    
    # 인용 기반 답변 생성 (간단한 예시)
    # 실제로는 관련 문서에서 직접 인용해야 함
    cited_answer = f"{answer} [관련 문서에서 인용]"
    
    # RAFT 훈련 템플릿
    text = f"질문: {question}\n\n문서들:\n{context_text}\n\n답변: {cited_answer}"
    
    return {
        "text": text,
        "question": question,
        "answer": cited_answer,
        "relevant_doc": relevant_doc,
        "distractor_docs": distractor_docs,
        "all_docs": all_docs
    }

# 테스트해보기
print("🧪 RAFT 훈련 샘플 생성 테스트:")
print("=" * 50)

# 첫 번째 샘플로 테스트
sample = dataset[0]
print("원본 데이터:")
print(f"질문: {sample['question']}")
print(f"답변: {sample['answer'][:100]}...")
print(f"관련 문서 개수: {len(sample['context'])}")
print(f"context 타입: {type(sample['context'])}")

# context가 문자열인지 확인
if isinstance(sample['context'], str):
    print(f"\n📝 Context는 문자열입니다. 길이: {len(sample['context'])}")
    print(f"문자열 미리보기: {sample['context'][:200]}...")
elif isinstance(sample['context'], list):
    print(f"\n📝 Context는 리스트입니다. 항목 수: {len(sample['context'])}")
    if sample['context']:
        print(f"첫 번째 항목: {sample['context'][0][:100]}...")

# 모든 문서 수집 (Distractor 생성을 위해)
print(f"\n📚 모든 문서 수집 중...")
all_contexts = []
for item in dataset:
    context = item['context']
    if isinstance(context, str):
        all_contexts.append(context)
    elif isinstance(context, list):
        if context and isinstance(context[0], str):
            all_contexts.extend(context)
        else:
            all_contexts.append("".join(context))

print(f"총 수집된 문서 개수: {len(all_contexts):,}개")

# RAFT 훈련 샘플 생성
raft_sample = create_raft_training_sample(
    sample['question'], 
    sample['answer'], 
    sample['context'],
    all_contexts,
    num_distractors=2
)

print(f"\n🎯 RAFT 훈련 샘플:")
print(f"관련 문서: 1개")
print(f"Distractor 문서 개수: {len(raft_sample['distractor_docs'])}")
print(f"전체 문서 개수: {len(raft_sample['all_docs'])}")
print(f"관련 문서 미리보기: {raft_sample['relevant_doc'][:200]}...")
print(f"Distractor 문서 미리보기: {raft_sample['distractor_docs'][0][:200]}...")
print(f"전체 내용: {raft_sample['text'][:500]}...")


## 4. 전체 데이터를 RAFT 형식으로 변환하기


In [ ]:
# 전체 데이터를 RAFT 형식으로 변환합니다 (Distractor Documents 포함!)
print("🔄 전체 데이터를 RAFT 형식으로 변환하는 중...")

# 모든 문서를 수집 (Distractor 생성을 위해)
print("📚 모든 문서 수집 중...")
all_contexts = []
for item in tqdm(dataset, desc="문서 수집"):
    context = item['context']
    if isinstance(context, str):
        all_contexts.append(context)
    else:
        # 리스트인 경우 문자열로 합치기
        all_contexts.append("".join(context))

print(f"✅ 총 {len(all_contexts):,}개의 문서 수집 완료!")

# 변환된 데이터를 저장할 리스트
raft_data = []

# 진행 상황을 보여주는 프로그레스 바로 변환
for i, sample in enumerate(tqdm(dataset, desc="RAFT 훈련 샘플 생성")):
    # RAFT 훈련 샘플 생성 (관련 문서 + Distractor 문서 + 인용 답변)
    raft_sample = create_raft_training_sample(
        sample['question'],
        sample['answer'], 
        sample['context'],
        all_contexts,
        num_distractors=2  # Distractor 문서 2개 추가
    )
    raft_data.append(raft_sample)

print(f"✅ RAFT 훈련 샘플 생성 완료!")
print(f"📊 생성된 샘플 개수: {len(raft_data):,}개")

# 샘플 확인
print(f"\n🔍 생성된 RAFT 샘플 예시:")
print(f"내용: {raft_data[0]['text'][:300]}...")
print(f"관련 문서: 1개")
print(f"Distractor 문서: {len(raft_data[0]['distractor_docs'])}개")


## 5. 변환된 데이터 통계 확인하기


In [ ]:
# 변환된 RAFT 데이터의 통계를 확인해봅시다
print("📊 RAFT 훈련 데이터 통계:")
print("=" * 50)

# 텍스트 길이 분석
text_lengths = [len(item['text']) for item in raft_data]

print(f"📝 텍스트 길이 분석:")
print(f"   평균: {np.mean(text_lengths):.1f} 글자")
print(f"   최소: {np.min(text_lengths)} 글자")
print(f"   최대: {np.max(text_lengths)} 글자")

# 문서 개수 분석
distractor_doc_counts = [len(item['distractor_docs']) for item in raft_data]
total_doc_counts = [len(item['all_docs']) for item in raft_data]

print(f"\n📄 문서 개수 분석:")
print(f"   관련 문서: 1개 (고정)")
print(f"   평균 Distractor 문서: {np.mean(distractor_doc_counts):.1f}개")
print(f"   평균 전체 문서: {np.mean(total_doc_counts):.1f}개")

# 길이 분포 확인
short_texts = sum(1 for length in text_lengths if length < 1000)
medium_texts = sum(1 for length in text_lengths if 1000 <= length < 2000)
long_texts = sum(1 for length in text_lengths if length >= 2000)

print(f"\n📏 길이별 분포:")
print(f"   짧은 텍스트 (<1000자): {short_texts:,}개 ({short_texts/len(raft_data)*100:.1f}%)")
print(f"   중간 텍스트 (1000-2000자): {medium_texts:,}개 ({medium_texts/len(raft_data)*100:.1f}%)")
print(f"   긴 텍스트 (≥2000자): {long_texts:,}개 ({long_texts/len(raft_data)*100:.1f}%)")

# 샘플 확인
print(f"\n🔍 RAFT 샘플 예시:")
sample = raft_data[0]
print(f"   질문: {sample['question']}")
print(f"   관련 문서: 1개")
print(f"   Distractor 문서 개수: {len(sample['distractor_docs'])}")
print(f"   전체 문서 개수: {len(sample['all_docs'])}")
print(f"   답변: {sample['answer']}")
print(f"   텍스트 길이: {len(sample['text'])} 글자")
print(f"   관련 문서 미리보기: {sample['relevant_doc'][:200]}...")
print(f"   전체 내용 미리보기: {sample['text'][:300]}...")


## 6. 파인튜닝용 데이터셋으로 변환하기


In [ ]:
# Hugging Face Dataset 형식으로 변환합니다
print("🔄 파인튜닝용 데이터셋으로 변환하는 중...")

# RAFT 데이터를 Hugging Face Dataset으로 변환
raft_dataset = Dataset.from_list(raft_data)

print(f"✅ 데이터셋 변환 완료!")
print(f"📊 최종 데이터셋 정보:")
print(f"   - 데이터 개수: {len(raft_dataset):,}개")
print(f"   - 필드: {list(raft_dataset.features.keys())}")
print(f"   - 첫 번째 샘플 키: {list(raft_dataset[0].keys())}")

# 데이터셋 구조 확인
print(f"\n🔍 데이터셋 구조:")
print(f"   - text: {raft_dataset.features['text']}")
print(f"   - question: {raft_dataset.features['question']}")
print(f"   - answer: {raft_dataset.features['answer']}")
print(f"   - relevant_doc: {raft_dataset.features['relevant_doc']}")
print(f"   - distractor_docs: {raft_dataset.features['distractor_docs']}")
print(f"   - all_docs: {raft_dataset.features['all_docs']}")


## 7. 최종 데이터 저장하기


In [ ]:
# 변환된 RAFT 데이터를 저장합니다
print("💾 RAFT 훈련 데이터를 저장하는 중...")

# 1. RAFT 데이터셋 저장 (다음 단계에서 사용)
raft_dataset.save_to_disk("data/raft_dataset")

# 2. 통계 정보 저장 (NumPy 타입을 Python 기본 타입으로 변환)
raft_stats = {
    "total_samples": int(len(raft_dataset)),
    "avg_text_length": float(np.mean(text_lengths)),
    "relevant_docs": 1,  # 고정값
    "avg_distractor_docs": float(np.mean(distractor_doc_counts)),
    "avg_total_docs": float(np.mean(total_doc_counts)),
    "min_text_length": int(np.min(text_lengths)),
    "max_text_length": int(np.max(text_lengths)),
    "short_texts": int(short_texts),
    "medium_texts": int(medium_texts),
    "long_texts": int(long_texts),
    "text_lengths": [int(length) for length in text_lengths]  # 리스트도 변환
}

with open("data/raft_stats.json", "w", encoding="utf-8") as f:
    json.dump(raft_stats, f, ensure_ascii=False, indent=2)

# 3. 샘플 데이터도 저장 (테스트용)
sample_data = raft_dataset.select(range(min(10, len(raft_dataset))))
sample_data.save_to_disk("data/raft_sample")

print("✅ 데이터 저장 완료!")
print("   - data/raft_dataset/: 전체 RAFT 훈련 데이터셋")
print("   - data/raft_sample/: 샘플 데이터 (10개)")
print("   - data/raft_stats.json: 통계 정보")


## 8. 다음 단계 안내

### 🎯 다음 노트북에서 할 일
**00.04-model-setup.ipynb**에서:
1. **EXAONE 모델** 로드하기
2. **토크나이저** 설정하기
3. **LoRA 설정**하기
4. **모델 준비** 완료하기

### 💡 지금까지 배운 것
- ✅ RAFT 방법론의 핵심 개념 이해
- ✅ Distractor Documents 포함한 훈련 데이터 생성
- ✅ "Open Book" 시험 환경 시뮬레이션
- ✅ 파인튜닝용 데이터셋 준비
- ✅ 데이터 통계 분석 및 저장

### 🔧 RAFT의 핵심 원리
- **"Open Book" 시험**: 모델이 문서를 참조하면서 답변하는 환경
- **Distractor Documents**: 관련 없는 방해 요소 문서들을 무시하도록 학습
- **인용 기반 답변**: 관련 문서에서 직접 인용하여 정확한 답변 생성
- **도메인 특화**: 특정 도메인의 문서 집합에 특화된 RAG 성능 향상

### 🎯 RAFT의 장점
- **문서 필터링**: 관련 있는 문서와 관련 없는 문서를 구분하는 능력 향상
- **인용 정확도**: 관련 문서에서 직접 인용하여 신뢰할 수 있는 답변 생성
- **도메인 적응**: 특정 도메인의 문서 집합에 특화된 성능 향상
- **RAG 견고성**: 검색의 불완전성에 대응하여 견고한 성능 발휘

### 🚀 준비 완료!
이제 다음 노트북으로 넘어가서 모델을 준비해보겠습니다!
